Base에서 실행

# Textcuboid


## 1) Dataload 20newsgroups  data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
x_train=list(X_train)
x_test=list(X_test)

특수문자 제거

In [7]:
import re
pattern = '[^a-z.0-9 ]'
clr_x_trian=[]
clr_x_test=[]
for sen in x_train:
    clr_x_trian.append(re.sub(pattern, '', sen))
for sen in x_test:
    clr_x_test.append(re.sub(pattern, '', sen))

In [8]:
x_train=clr_x_trian
x_test=clr_x_test

In [9]:
to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [10]:
#불용어 불러오기
import os
os.chdir('..')
with open('./english_stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [11]:
stopwords=stopwords+['aaa', 'ab']

In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt_filter)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names())
sw=word_view[TorF]

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
#선별된 10000개 단어
sw

array(['aa', 'aamir', 'aap', ..., 'zy', 'zyxel', 'zz'], dtype='<U208')

In [14]:
feature_lst10000=sw

## 2) textcuboid 생성

In [15]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [16]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [17]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(15062):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Train_data에서 가장 많은 feature를 가진 문서의 경우 feature 952  개를 가짐
Train_data에서 가장 적은 feature를 가진 문서의 경우 feature 0  개를 가짐


In [18]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(3766):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Test_data에서 가장 많은 feature를 가진 문서의 경우 feature 923  개를 가짐
Test_data에서 가장 적은 feature를 가진 문서의 경우 feature 0  개를 가짐


In [21]:
#1-Channel TextCuboid 생성
textcuboid=[]

for i in range(15062):
    frame1=np.zeros((952,256))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            frame1[idx_cnt]=np.load('./elmo_embedding/train(20ng256)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid.append(frame1)

textcuboid=np.array(textcuboid)

np.save('./1-Channel textcuboid_20ng(elmo).npy',textcuboid)

In [22]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

for i in range(3766):
    frame1=np.zeros((952,256))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            frame1[idx_cnt]=np.load('./elmo_embedding/test(20ng256)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid_test.append(frame1)

textcuboid_test=np.array(textcuboid_test)
np.save('./1-Channel textcuboid_test_20ng(elmo).npy',textcuboid_test)